In [1]:
def is_colab():
    try:
        from google.colab import drive
        return True
    except:
        return False

In [2]:

if is_colab():
    !git clone https://github.com/Priyanshu-hawk/pytorch_random.git

In [3]:
import os
if is_colab() and os.getcwd() != '/content/pytorch_random': #vauge but works
    %cd /content/pytorch_random

In [4]:
!pip install -r req.txt
!pip install requests

In [ ]:
import requests

if not os.path.exists("../Modular_DeepLearning/food101_3_class/data"):
    with open("../Modular_DeepLearning/food101_3_class/data.zip", "wb") as f:
        r = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/refs/heads/main/data/pizza_steak_sushi.zip")
        f.write(r.content)

    import zipfile, os
    with zipfile.ZipFile("../Modular_DeepLearning/food101_3_class/data.zip", "r") as zip_ref:
        os.makedirs("../Modular_DeepLearning/food101_3_class/data", exist_ok=True)
        zip_ref.extractall("../Modular_DeepLearning/food101_3_class/data")

In [ ]:
import os
data = "../Modular_DeepLearning/food101_3_class/data"
train_path = os.path.join(data,"train")
test_path = os.path.join(data,"test")

In [7]:
from Modular_DeepLearning.food101_3_class.data_setup import createDataloader
import torchvision, torch

In [8]:
model_weights = torchvision.models.ResNet101_Weights.DEFAULT
_model = torchvision.models.resnet101(pretrained=model_weights)

/home/infinity/Desktop/Learn Pytorch/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/infinity/Desktop/Learn Pytorch/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
auto_transform = model_weights.transforms()
auto_transform

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [17]:
train_load, test_load, class_names = createDataloader(train_path, test_path, BTACH_SIZE=32, train_transform=auto_transform, test_transform=auto_transform)

In [19]:
from torchinfo import summary
summary(_model, input_size=(32, 3,224,224), col_names=("input_size", "output_size", "num_params", "trainable"),
        row_settings=["var_names"])

Layer (type (var_name))                  Input Shape               Output Shape              Param #                   Trainable
ResNet (ResNet)                          [32, 3, 224, 224]         [32, 1000]                --                        True
├─Conv2d (conv1)                         [32, 3, 224, 224]         [32, 64, 112, 112]        9,408                     True
├─BatchNorm2d (bn1)                      [32, 64, 112, 112]        [32, 64, 112, 112]        128                       True
├─ReLU (relu)                            [32, 64, 112, 112]        [32, 64, 112, 112]        --                        --
├─MaxPool2d (maxpool)                    [32, 64, 112, 112]        [32, 64, 56, 56]          --                        --
├─Sequential (layer1)                    [32, 64, 56, 56]          [32, 256, 56, 56]         --                        True
│    └─Bottleneck (0)                    [32, 64, 56, 56]          [32, 256, 56, 56]         --                        True
│    │ 

In [20]:
# frezzing resnet layer which learn on images
frezing_layers = [_model.layer1, _model.layer2, _model.layer3, _model.layer4]
for fl in frezing_layers:
    for prams in fl.parameters():
        prams.requires_grad = False

In [21]:
_model.fc = torch.nn.Linear(in_features=2048, out_features=len(class_names), bias=True)

In [22]:
_model = _model.to("cuda")

In [23]:
from torchinfo import summary
summary(_model, input_size=(32, 3,224,224), col_names=("input_size", "output_size", "num_params", "trainable"),
        row_settings=["var_names"])

Layer (type (var_name))                  Input Shape               Output Shape              Param #                   Trainable
ResNet (ResNet)                          [32, 3, 224, 224]         [32, 3]                   --                        Partial
├─Conv2d (conv1)                         [32, 3, 224, 224]         [32, 64, 112, 112]        9,408                     True
├─BatchNorm2d (bn1)                      [32, 64, 112, 112]        [32, 64, 112, 112]        128                       True
├─ReLU (relu)                            [32, 64, 112, 112]        [32, 64, 112, 112]        --                        --
├─MaxPool2d (maxpool)                    [32, 64, 112, 112]        [32, 64, 56, 56]          --                        --
├─Sequential (layer1)                    [32, 64, 56, 56]          [32, 256, 56, 56]         --                        False
│    └─Bottleneck (0)                    [32, 64, 56, 56]          [32, 256, 56, 56]         --                        False
│ 

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
loss_fn  = torch.nn.CrossEntropyLoss().to("cuda").to(device)
optimizer = torch.optim.Adam(_model.parameters(), lr=0.001)

In [26]:
try:
    from torch.utils.tensorboard import SummaryWriter
except:
    !pip install tensorboard
    from torch.utils.tensorboard import SummaryWriter

writter = SummaryWriter()
writter

In [27]:
def acc_func(y_preds, y):
    # print(y_preds)
    acc_score = torch.eq(y_preds, y).sum() / len(y)
    return acc_score

In [28]:
from tqdm import tqdm

In [29]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [31]:
def train(model, loss_fn, optim, train_dataloader):
    train_acc = 0
    train_loss = 0
    model.train()
    for x, y in train_dataloader:
        x=x.to(device)
        y=y.to(device)
        y_logits = model(x)
        loss = loss_fn(y_logits, y)
        train_loss+=loss.item()
        train_acc+=acc_func(torch.argmax(y_logits, dim=1), y)
        optim.zero_grad()
        loss.backward()
        optim.step()

    return train_loss/len(train_dataloader), train_acc/len(train_dataloader)

def test(model, loss_fn, test_dataloader):
    model.eval()
    test_loss = 0
    test_acc = 0
    for x, y in test_dataloader:
        x=x.to(device)
        y=y.to(device)
        y_logits = model(x)
        loss = loss_fn(y_logits, y)
        test_loss+=loss
        test_acc += acc_func(torch.argmax(y_logits, dim=1), y)
        loss.backward()

    return test_loss/len(test_dataloader) , test_acc/len(test_dataloader)



train_data = {
        "Acc": [],
        "Loss": []
    }

test_data = {
    "Acc": [],
    "Loss": []
}

epochs = 10
cala_test_loss_acc = True

for e in tqdm(range(epochs)):
    train_loss, train_acc = train(_model, loss_fn, optimizer, train_load)
    print("Train: Loss - {} | Accuracy - {}".format(train_loss, train_acc))

    train_data["Acc"].append(train_acc)
    train_data["Loss"].append(train_loss)

    if cala_test_loss_acc:
        test_loss, test_acc = test(_model, loss_fn, test_load)
        print("Test: Loss - {} | Accuracy - {}".format(test_loss, test_acc))

        test_data["Acc"].append(test_acc)
        test_data["Loss"].append(test_loss)

    writter.add_scalar("Loss/Train", scalar_value=train_loss, global_step=e)
    writter.add_scalar("Loss/Test", test_loss, e)
    writter.add_scalar("Acc/Train", train_acc, e)
    writter.add_scalar("Acc/Test", test_acc, e)

    writter.add_graph(_model, input_to_model=torch.randn(32, 3, 224, 224).to(device))

writter.close()




  0%|          | 0/10 [00:00<?, ?it/s]

Train: Loss - 0.5766129376183093 | Accuracy - 0.7737675905227661
Test: Loss - 0.2726518213748932 | Accuracy - 0.9283854365348816


 10%|█         | 1/10 [00:25<03:45, 25.03s/it]

Train: Loss - 0.3030552775926993 | Accuracy - 0.8961267471313477
Test: Loss - 0.18996034562587738 | Accuracy - 0.9475446939468384


 20%|██        | 2/10 [00:49<03:16, 24.53s/it]

...

 80%|████████  | 8/10 [03:12<00:47, 23.88s/it]

Train: Loss - 0.16067857688791315 | Accuracy - 0.9414612650871277
Test: Loss - 0.11941727250814438 | Accuracy - 0.9609375


 90%|█████████ | 9/10 [03:36<00:23, 23.79s/it]

Train: Loss - 0.14844421145152037 | Accuracy - 0.9480633735656738
Test: Loss - 0.1136355847120285 | Accuracy - 0.9622396230697632


100%|██████████| 10/10 [03:59<00:00, 23.97s/it]


In [ ]:
# test with few settings like
# epochs = 5, 10
# models = resnet18, resnet50, resnet101
# optimizer = Adam, SGD
# learning rate = 0.001, 0.01
# try with different data augmentation